In [48]:
# Creating a user interface so that the user can enter 
# their own initial state for the puzzle.
import math
MAX_SIZE = 100

def main():
    # TODO: Right now it fails if user make mistakes
    # Check size to be an integer square - 1, use try except on rows input
    square_less_1 = [x**2-1 for x in range(2, 11, 1)]
    size = int(input("This is an N-puzzle solver. Please choose size of game " \
                     f"(can only be N^2-1, e.g. {square_less_1}) no larger than {MAX_SIZE}:"))
    if size > MAX_SIZE:
        print("Size is too large, please try again")
        return
    if size not in square_less_1:
        print('Size game entered is not an integer square - 1, please try again')
        return
    N = int(math.sqrt(size+1))
    puzzle_type = input("Press 1 to create your own puzzle or 2 for a default puzzle:")
    if puzzle_type == '1':
        print("When entering your puzzle, make sure to enter 0 for the blank space and use spaces between numbers:")
        # Creating an empty NxN matrix for the user to create the initial state
        try:
            mat = [N*[0] for i in range(N)]
            # Iterating over each row in the matrix
            for i in range(N):
                mat[i] = input(f"Enter row {str(i)}: ")
            #print(mat)
            # Since the input the user gives for the puzzle is a string,
            # we need to convert the elements of this puzzle to integers.
            for j in range(N):
                splt_row = mat[j].split()
                mat[j] = [int(splt_row[k]) for k in range(N)]
#                 print(mat[j])
        except:
            print("Please pay attention to the format required")
            return
    if puzzle_type == '2':
        pass
        # call function that will run search with default puzzle. 
        # maybe give options for the difficulty of default puzzle.
        
    print(f"This is the starting state:")
    for i in range(N):
        print(mat[i])
        
    # Ask user for:
    # Algorithm
    # Would you like a print of the best sequence of moves y/n - into PRINT_MOVES
    # Would you like a full trace of the algorithm? NOTE: this could be pretty long - into PRINT_TRACE

In [49]:
main()

This is an N-puzzle solver. Please choose size of game (can only be N^2-1, e.g. [3, 8, 15, 24, 35, 48, 63, 80, 99]) no larger than 100:8
Press 1 to create your own puzzle or 2 for a default puzzle:1
When entering your puzzle, make sure to enter 0 for the blank space and use spaces between numbers:
Enter row 0: 1 2 3
Enter row 1: 1 2 3
Enter row 2: 1 2 3
This is the starting state:
[1, 2, 3]
[1, 2, 3]
[1, 2, 3]


In [ ]:
s = '1 1 1'
[int(s.split()[i]) for i in range(3)]

In [ ]:
a = input("what option?")

In [ ]:
[N*[0] for i in range(N)]

In [ ]:
# Creating a function that will create the goal state based
# on the number of dimensions that the user gives it.
# Arguments:
# N: The number of rows and columns for the square matrix.
def puzzle(N):
    # Creating an matrix of dimension NxN with all 0's
    mat = [N*[0] for i in range(N)]
    # Iterating over all rows
    for r in range(N):
        # For each row, iterating over each column
        for c in range(N):
            #print(r,c)
            # Assigning the corresponding value of the puzzle
            # by using the following formula.
            mat[r][c] = N*r + c%N + 1
            
    # Based on the formula used for each element above, the [N,N] position
    # will contain (N+1) but we want it to be 0 to represent an empty space
    mat[N-1][N-1] = 0
    
    return mat

In [ ]:
puzzle(2)

In [ ]:
a = puzzle(3)
a[1]

In [ ]:
import copy
b = copy.deepcopy(a)
b[0][0]=100
b

In [ ]:
def mm(mat_src):
    mat = copy.deepcopy(mat_src)
    mat[0][0] = 200
    return mat

In [ ]:
b = mm(a)

In [ ]:
a

In [ ]:
b

In [ ]:
# This function creates a dictionary with the correct indices for each of 
# the tiles in the given state.
def mat2dict(mat):
    # Defining an intial empty dictiory
    goal_dict = {}
    # The dimension of mat
    n = len(mat)
    # Iterating over each element in the matrix
    for i in range(n):
        for j in range(n):
            # The value of the i,j tile
            tile = mat[i][j]
            #print(tile)
            # For each tile, assign its corresponding indices
            goal_dict[tile] = [i,j]
    
    return goal_dict

In [ ]:
# Testing the function mat2dict
mat2dict(puzzle(2))

In [ ]:
10//3

In [ ]:
# Creating a function for the A* algorithm that will work for 
# uniform cost search, the misplaced tile heuristic and the 
# manhatten distance heuristic.
# The arguments for the function are:
# goal_mat: The goal state for the puzzle
# start_mat: The puzzle that we are starting the search with.
# g: A cost function
# h: A heuristic function
def astar(goal_mat, start_mat, g, h):
    # defining the cost that w

In [ ]:
def h_0(crnt, goal):
    return 0

In [ ]:
def h_misplace(crnt, goal):
    # number of rows and columns in crnt
    n = len(crnt)
    # Counter for the number of misplaced tiles
    misplace = 0
    # iterating over all elements in the crnt and goal matrices
    for i in range(n):
        for j in range(n):
            print(crnt[i][j], goal[i][j])
            # Checking to see if the i,j element in crnt state 
            # is in the right place or not. Including a condition
            # that the misplaced element can't equal 0 since this
            # heuristic doesn't consider it as misplaced.
            if crnt[i][j] != goal[i][j] and crnt[i][j] != 0:
                # Each time we have a misplaced tile, misplace increases by 1
                misplace += 1
    return misplace

In [ ]:
state = [[1,2,4],[3,0,6],[7,8,5]]
h_misplace(state, puzzle(3))

In [ ]:
def h_manhatt(crnt, goal):
    # Dimension of crnt and goal
    n = len(crnt)
    # Creating a variable for the manhattan distance
    manhatt_dist = 0
    # Iterating over all elements in the matrix
    for i in range(n):
        for j in range(n):
            # Don't want to calculate manhattan distance for tile 0
            if crnt[i][j] != 0:
                # The value of the i,j tile in crnt
                tile = crnt[i][j]
                # The correct indices based on the goal state
                # for the value tile
                crct_ind = mat2dict(goal)[tile]
                # Calculating the manhattan distance
                manhatt_dist += abs(i - crct_ind[0]) + abs(j - crct_ind[1])
    return manhatt_dist

In [ ]:
# Testing h_manhatt
state = [[1,2,3],[5,4,6],[7,8,0]]
h_manhatt(state, puzzle(3))
#state

In [ ]:
# This function determines the next possible moves based on the 
# current state in the search.
def next_move(crnt):
    # Creating a dictionary of pieces and their corresponding indices
    # so that we can identify the indices of 0, the blank space.
    crnt_dict = mat2dict(crnt)
    # The indices of the blank space 0
    blank_ind = crnt_dict[0]

In [ ]:
# Function that determines whether or not the search has 
# reached the goal state given the current state.
# Arguments:
# crnt: The current state in the search.
# goal: the goal state of the search.
# Return:
# 
def goal_state(crnt, goal):
    # Compares the crnt and goal matrices to determine if they 
    # are equivalent.
    return(crnt == goal)

In [ ]:
mat = [[1,0],[0,1]]
mat[0]

In [ ]:
abs(44)

In [ ]:
N=3
mat = [3*[0] for i in range(N)]
mat

In [ ]:
mat[0][0] = 1
mat[2][2] = 4
mat

In [ ]:
mat1 = [3*[0] for i in range(N)]
mat1

In [ ]:
mat1 == mat

In [ ]:
mat[0][0] = 12

In [ ]:
mat1

In [ ]:
cars = ['kia', 'audi', 'bmw']
for car in enumerate(cars):
  print(car)

In [ ]:
dic = {1:[0,0], 2:[0,1]}

In [ ]:
dic[1] = 1

In [ ]:
dic